In [28]:
from gurobipy import Model, GRB
import pandas as pd

In [29]:
# Load the datasets
# Note: Replace the paths with the actual paths where you have saved your datasets
capacity_direct_prod_facilities = pd.read_csv('/Users/mahinbindra/Downloads/Capacity_for_Direct_Production_Facilities (1).csv')
capacity_transship_dist_centers = pd.read_csv('/Users/mahinbindra/Downloads/Capacity_for_Transship_Distribution_Centers (1).csv')
capacity_transship_prod_facilities = pd.read_csv('/Users/mahinbindra/Downloads/Capacity_for_Transship_Production_Facilities (1).csv')
cost_prod_to_refinement = pd.read_csv('/Users/mahinbindra/Downloads/Cost_Production_to_Refinement (1).csv')
cost_prod_to_transshipment = pd.read_csv('/Users/mahinbindra/Downloads/Cost_Production_to_Transshipment (1).csv')
cost_transshipment_to_refinement = pd.read_csv('/Users/mahinbindra/Downloads/Cost_Transshipment_to_Refinement (1).csv')
refinement_demand = pd.read_csv('/Users/mahinbindra/Downloads/Refinement_Demand (2).csv')

In [30]:
# Initialize the model
m = Model("Can2Oil_Optimization")

In [31]:
# Decision Variables
x = m.addVars(cost_prod_to_refinement.ProductionFacility.unique(), cost_prod_to_refinement.RefinementCenter.unique(), name="x", lb=0)
y = m.addVars(cost_prod_to_transshipment.ProductionFacility.unique(), cost_prod_to_transshipment.TransshipmentHub.unique(), name="y", lb=0)
z = m.addVars(cost_transshipment_to_refinement.TransshipmentHub.unique(), cost_transshipment_to_refinement.RefinementCenter.unique(), name="z", lb=0)

In [32]:
# Corrected Objective Function
m.setObjective(
    sum(x[i,j] * cost_prod_to_refinement.loc[(cost_prod_to_refinement['ProductionFacility'] == i) & (cost_prod_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for i in cost_prod_to_refinement['ProductionFacility'].unique()
        for j in cost_prod_to_refinement['RefinementCenter'].unique()) +
    sum(y[i,k] * cost_prod_to_transshipment.loc[(cost_prod_to_transshipment['ProductionFacility'] == i) & (cost_prod_to_transshipment['TransshipmentHub'] == k), 'Cost'].values[0]
        for i in cost_prod_to_transshipment['ProductionFacility'].unique()
        for k in cost_prod_to_transshipment['TransshipmentHub'].unique()) +
    sum(z[k,j] * cost_transshipment_to_refinement.loc[(cost_transshipment_to_refinement['TransshipmentHub'] == k) & (cost_transshipment_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for k in cost_transshipment_to_refinement['TransshipmentHub'].unique()
        for j in cost_transshipment_to_refinement['RefinementCenter'].unique()),
    GRB.MINIMIZE
)


In [33]:
# Constraints

# Production Capacity Constraints
for i in capacity_direct_prod_facilities.ProductionFacility:
    m.addConstr(sum(x[i,j] for j in cost_prod_to_refinement.RefinementCenter.unique()) <= capacity_direct_prod_facilities.loc[capacity_direct_prod_facilities.ProductionFacility==i, 'Capacity'].values[0])

for i in capacity_transship_prod_facilities.ProductionFacility:
    m.addConstr(sum(y[i,k] for k in cost_prod_to_transshipment.TransshipmentHub.unique()) <= capacity_transship_prod_facilities.loc[capacity_transship_prod_facilities.ProductionFacility==i, 'Capacity'].values[0])

# Distribution Center Capacity Constraints
for k in capacity_transship_dist_centers.TransshipmentHub:
    m.addConstr(sum(y[i,k] for i in cost_prod_to_transshipment.ProductionFacility.unique()) == sum(z[k,j] for j in cost_transshipment_to_refinement.RefinementCenter.unique()))
    m.addConstr(sum(z[k,j] for j in cost_transshipment_to_refinement.RefinementCenter.unique()) <= capacity_transship_dist_centers.loc[capacity_transship_dist_centers.TransshipmentHub==k, 'Capacity'].values[0])

# Demand Constraints
for j in refinement_demand.RefinementCenter:
    m.addConstr(sum(x[i,j] for i in cost_prod_to_refinement.ProductionFacility.unique()) + sum(z[k,j] for k in cost_transshipment_to_refinement.TransshipmentHub.unique()) == refinement_demand.loc[refinement_demand.RefinementCenter==j, 'Demand'].values[0])

In [34]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 340 nonzeros
Model fingerprint: 0x5f5fd4bd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 340 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   1.238600e+04   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [35]:
# Display the results
if m.status == GRB.OPTIMAL:
    solution_x = m.getAttr('x', x)
    solution_y = m.getAttr('x', y)
    solution_z = m.getAttr('x', z)
    print("Optimal solution found:")
    print("Direct shipments (Production to Refinement):", solution_x)
    print("Transshipment (Production to Distribution Centers):", solution_y)
    print("Distribution to Refinement:", solution_z)
else:
    print("Optimal solution was not found.")

Optimal solution found:
Direct shipments (Production to Refinement): {(1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 462.0, (1, 5): 0.0, (2, 1): 0.0, (2, 2): 103.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 460.0, (3, 4): 0.0, (3, 5): 0.0, (4, 1): 0.0, (4, 2): 0.0, (4, 3): 0.0, (4, 4): 0.0, (4, 5): 0.0, (5, 1): 0.0, (5, 2): 0.0, (5, 3): 0.0, (5, 4): 86.0, (5, 5): 0.0, (6, 1): 0.0, (6, 2): 217.0, (6, 3): 0.0, (6, 4): 0.0, (6, 5): 0.0, (7, 1): 0.0, (7, 2): 0.0, (7, 3): 0.0, (7, 4): 0.0, (7, 5): 0.0, (8, 1): 0.0, (8, 2): 0.0, (8, 3): 0.0, (8, 4): 0.0, (8, 5): 521.0, (9, 1): 0.0, (9, 2): 0.0, (9, 3): 0.0, (9, 4): 0.0, (9, 5): 548.0, (10, 1): 0.0, (10, 2): 0.0, (10, 3): 0.0, (10, 4): 0.0, (10, 5): 0.0, (11, 1): 0.0, (11, 2): 0.0, (11, 3): 0.0, (11, 4): 0.0, (11, 5): 354.0, (12, 1): 7.0, (12, 2): 0.0, (12, 3): 404.0, (12, 4): 0.0, (12, 5): 0.0, (13, 1): 104.0, (13, 2): 0.0, (13, 3): 0.0, (13, 4): 0.0, (13, 5): 0.0, (14, 1): 0.0, (14, 2): 0.0, (14, 3): 0.0, (14,

In [36]:
# Calculate total transshipped
total_transshipped = sum(y[i,k].X for i in cost_prod_to_transshipment['ProductionFacility'].unique() for k in cost_prod_to_transshipment['TransshipmentHub'].unique())

# Calculate total shipped directly and through transshipment
total_shipped = sum(x[i,j].X for i in cost_prod_to_refinement['ProductionFacility'].unique() for j in cost_prod_to_refinement['RefinementCenter'].unique()) + sum(z[k,j].X for k in cost_transshipment_to_refinement['TransshipmentHub'].unique() for j in cost_transshipment_to_refinement['RefinementCenter'].unique())

# Calculate the proportion of canola oil that is transshipped
proportion_transshipped = total_transshipped / total_shipped if total_shipped > 0 else 0

print("Proportion of canola oil transshipped:", proportion_transshipped)

Proportion of canola oil transshipped: 0.31736938588450964


In [37]:
P = 1  # Example penalty cost per million pounds transshipped. Adjust this value based on the desired penalty.

# Modified Objective Function with Transshipment Penalty
m.setObjective(
    sum(x[i,j] * cost_prod_to_refinement.loc[(cost_prod_to_refinement['ProductionFacility'] == i) & (cost_prod_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for i in cost_prod_to_refinement['ProductionFacility'].unique()
        for j in cost_prod_to_refinement['RefinementCenter'].unique()) +
    sum(y[i,k] * (cost_prod_to_transshipment.loc[(cost_prod_to_transshipment['ProductionFacility'] == i) & (cost_prod_to_transshipment['TransshipmentHub'] == k), 'Cost'].values[0] + P)
        for i in cost_prod_to_transshipment['ProductionFacility'].unique()
        for k in cost_prod_to_transshipment['TransshipmentHub'].unique()) +
    sum(z[k,j] * cost_transshipment_to_refinement.loc[(cost_transshipment_to_refinement['TransshipmentHub'] == k) & (cost_transshipment_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for k in cost_transshipment_to_refinement['TransshipmentHub'].unique()
        for j in cost_transshipment_to_refinement['RefinementCenter'].unique()),
    GRB.MINIMIZE
)

# Optimize the model
m.optimize()

# After optimization, extract and analyze the solution as needed
if m.status == GRB.OPTIMAL:
    # Example: Print the total amount transshipped
    total_transshipped = sum(y[i,k].X for i in cost_prod_to_transshipment['ProductionFacility'].unique() for k in cost_prod_to_transshipment['TransshipmentHub'].unique())
    print("Total amount transshipped:", total_transshipped)
    # Further analysis as required

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 340 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.6737895e+29   1.000000e+31   6.673789e-01      0s
       5    2.6862012e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.686201173e+04
Total amount transshipped: 2618.0


In [38]:
# New constraint to limit the total amount of canola oil that is transshipped
# Define the maximum allowed transshipment volume
max_transshipment_volume = 5000  # Adjust this value as needed

# Constraint to limit the total transshipment volume
transshipment_volume_constraint = m.addConstr(sum(y[i,k] for i in cost_prod_to_transshipment['ProductionFacility'].unique() for k in cost_prod_to_transshipment['TransshipmentHub'].unique()) <= max_transshipment_volume, "Max_Transshipment_Volume")

# Solve the model
m.optimize()

# Extract and display the solution
if m.status == GRB.OPTIMAL:
    # Print the total amount transshipped and check it against the limit
    total_transshipped = sum(y[i,k].X for i in cost_prod_to_transshipment['ProductionFacility'].unique() for k in cost_prod_to_transshipment['TransshipmentHub'].unique())
    print(f"Total amount transshipped: {total_transshipped}, within the limit of {max_transshipment_volume}")
    # Additional analysis as required
else:
    print("Optimal solution was not found.")

# if m.status == GRB.Status.OPTIMAL:
#     print('Optimal solution found with total cost:', m.objVal)
#     for v in m.getVars():
#         if v.x > 0:
#             print(f'{v.varName} = {v.x}')
# else:
#     print('No optimal solution found')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 165 columns and 370 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6862012e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.686201173e+04
Total amount transshipped: 2618.0, within the limit of 5000


In [40]:
# Define a discount for producers closer to North America (Canada, US, Mexico)
discount_factor = 0.9  # For example, a 10% discount on transportation costs

# Apply the discount to the costs from production facilities in Canada, US, and Mexico
for index, row in cost_prod_to_refinement.iterrows():
    if row['ProductionFacility'] <= 15:  # Assuming facilities 1-15 are in Canada, US, and Mexico
        cost_prod_to_refinement.at[index, 'Cost'] *= discount_factor

# Objective Function - Assuming it's defined similarly, but now with possibly adjusted costs
m.setObjective(
    sum(x[i,j] * cost_prod_to_refinement.loc[(cost_prod_to_refinement['ProductionFacility'] == i) & (cost_prod_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for i in cost_prod_to_refinement['ProductionFacility'].unique()
        for j in cost_prod_to_refinement['RefinementCenter'].unique()) +
    sum(y[i,k] * cost_prod_to_transshipment.loc[(cost_prod_to_transshipment['ProductionFacility'] == i) & (cost_prod_to_transshipment['TransshipmentHub'] == k), 'Cost'].values[0]
        for i in cost_prod_to_transshipment['ProductionFacility'].unique()
        for k in cost_prod_to_transshipment['TransshipmentHub'].unique()) +
    sum(z[k,j] * cost_transshipment_to_refinement.loc[(cost_transshipment_to_refinement['TransshipmentHub'] == k) & (cost_transshipment_to_refinement['RefinementCenter'] == j), 'Cost'].values[0]
        for k in cost_transshipment_to_refinement['TransshipmentHub'].unique()
        for j in cost_transshipment_to_refinement['RefinementCenter'].unique()),
    GRB.MINIMIZE
)

# Optimize the model
m.optimize()

# Display the results
if m.status == GRB.OPTIMAL:
    solution_x = m.getAttr('x', x)
    solution_y = m.getAttr('x', y)
    solution_z = m.getAttr('x', z)
    print("Optimal solution found:")
    # Example of how to print some results
    for i, j in solution_x.keys():
        if solution_x[i, j] > 0:
            print(f"Facility {i} to Refinement {j}: {solution_x[i,j]} millions of pounds")
else:
    print("Optimal solution was not found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 165 columns and 370 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.1281398e+28   5.000000e+30   3.128140e-02      0s
       3    2.3049655e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.304965494e+04
Optimal solution found:
Facility 1 to Refinement 4: 462.0 millions of pounds
Facility 2 to Refinement 2: 103.0 millions of pounds
Facility 3 to Refinement 3: 460.0 millions of pounds
Facility 5 to Refinement 4: 86.0 millions of pounds
Facility 6 to Refinement 2: 217.0 millions of pound